In [3]:
import json
import os
import bs4 as bs
import sys
import re
import time
import requests
from lxml import etree
from lxml import html
from unidecode import unidecode
from lxml.etree import tostring
from itertools import chain
from unidecode import unidecode
from urllib.request import urlopen
import pandas as pd
import numpy as np
from xml.etree import ElementTree as et

from ftplib import FTP
import io 
import boto3
import glob
import shutil

Perform the query on PubMed using the randomized clinical trials search string [Reference for search in Medline](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1435857/)  [Cochrane Work](http://work.cochrane.org/pubmed)  and [Overviews of performance](https://sites.google.com/a/york.ac.uk/issg-search-filters-resource/filters-to-identify-randomized-controlled-trials-and)

In [4]:
#query = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmax=300000&term=(Clinical%20Trial%5BPublication%20Type%5D)%20AND%20Free%20full%20text%5BFilter%5D"
#query = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&retmax=300000&term=(Randomized%20Controlled%20Trial%5BPublication%20Type%5D)"

query = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmax=4659616&term=((randomized+controlled+trial%5Bpt%5D)+OR+(controlled+clinical+trial%5Bpt%5D)+OR+(randomized%5Btiab%5D+OR+randomised%5Btiab%5D)+OR+(placebo%5Btiab%5D)+OR+(drug+therapy%5Bsh%5D)+OR+(randomly%5Btiab%5D)+OR+(trial%5Btiab%5D)+OR+(groups%5Btiab%5D))+NOT+(animals%5Bmh%5D+NOT+humans%5Bmh%5D)"

resp = urlopen(query).read()#.decode('utf-8')

soup = bs.BeautifulSoup(resp,'lxml')
targetIDs = soup.find_all('id')
small_list = []
for docID in targetIDs:
    small_list.append(docID.get_text())
    #print(docID.get_text())

Check how many articles are returned in the PubMed query

In [5]:
print(len(small_list))

3664349


Open the OpenAccess file list from the downloaded CSV file, so that we can match PMID against PMCID

In [6]:
openaccess_df = pd.read_csv('oa_file_list.csv')

print(openaccess_df.shape)

openaccess_df.head(10)

(1875205, 6)


,File,Article Citation,Accession ID,Last Updated (YYYY-MM-DD HH:MM:SS),PMID,License
0,oa_package/08/e0/PMC13900.tar.gz,Breast Cancer Res. 2001 Nov 2; 3(1):55-60,PMC13900,2017-04-26 12:15:50,11250746.0,NO-CC CODE
1,oa_package/b0/ac/PMC13901.tar.gz,Breast Cancer Res. 2001 Nov 9; 3(1):61-65,PMC13901,2016-01-20 10:58:46,11250747.0,NO-CC CODE
2,oa_package/f7/98/PMC13902.tar.gz,Breast Cancer Res. 2001 Nov 8; 3(1):66-75,PMC13902,2006-02-02 19:37:52,11250748.0,NO-CC CODE
3,oa_package/9c/7f/PMC13911.tar.gz,Breast Cancer Res. 2000 Nov 16; 2(1):59-63,PMC13911,2013-03-17 14:00:52,11056684.0,NO-CC CODE
4,oa_package/c6/fb/PMC13912.tar.gz,Breast Cancer Res. 2000 Dec 6; 2(1):64-72,PMC13912,2013-03-17 14:00:52,11400682.0,NO-CC CODE
5,oa_package/3b/77/PMC13913.tar.gz,Breast Cancer Res. 1999 Aug 19; 1(1):73-80,PMC13913,2013-03-17 14:00:52,11056681.0,NO-CC CODE
6,oa_package/4b/13/PMC13914.tar.gz,Breast Cancer Res. 1999 Sep 1; 1(1):81-87,PMC13914,2013-03-17 14:00:52,11056682.0,NO-CC CODE
7,oa_package/cb/d1/PMC13915.tar.gz,Breast Cancer Res. 1999 Oct 7; 1(1):88-94,PMC13915,2013-03-17 14:00:52,11056683.0,NO-CC CODE
8,oa_package/1e/3b/PMC13916.tar.gz,Breast Cancer Res. 2000 Feb 21; 2(2):139-148,PMC13916,2014-02-18 06:06:35,11056686.0,NO-CC CODE
9,oa_package/0e/7e/PMC13917.tar.gz,Breast Cancer Res. 2000 Mar 7; 2(3):222-235,PMC13917,2013-05-17 12:53:06,11056687.0,NO-CC CODE


Get the subset of RCT articles that are in PMC; print out the number of them

In [7]:
clinical_oa_df = openaccess_df[openaccess_df['PMID'].isin(small_list)]

print(clinical_oa_df.shape)

(291249, 6)


Save the resulting dataframe to CSV file

In [8]:
clinical_oa_df.to_csv('clinical_oa_df.csv')
#clinical_oa_df = pd.read_csv('clinical_oa_df.csv')
#clinical_oa_df = clinical_oa_df.reset_index(drop=True)

In [9]:
clinical_oa_df.head()

,File,Article Citation,Accession ID,Last Updated (YYYY-MM-DD HH:MM:SS),PMID,License
13,oa_package/17/bb/PMC13921.tar.gz,Breast Cancer Res. 2000 Aug 21; 2(6):438-443,PMC13921,2014-04-29 14:49:00,11056691.0,NO-CC CODE
15,oa_package/75/59/PMC13923.tar.gz,Breast Cancer Res. 2001 Dec 22; 3(2):122-133,PMC13923,2014-04-29 19:39:41,11250759.0,NO-CC CODE
24,oa_package/9c/ed/PMC15027.tar.gz,Genome Biol. 2000 Nov 6; 1(5):research0009.1-r...,PMC15027,2014-04-29 19:45:12,11178258.0,NO-CC CODE
29,oa_package/be/ab/PMC16145.tar.gz,Genome Biol. 2000 Dec 4; 1(6):research0014.1-1...,PMC16145,2014-04-29 19:46:09,11178268.0,NO-CC CODE
39,oa_package/70/ed/PMC17806.tar.gz,Arthritis Res. 2000 Dec 22; 2(1):75-84,PMC17806,2014-04-29 19:51:28,11219392.0,NO-CC CODE


Get the target list of PMCIDs that will be used to reference article XML files

Export the target list of PMCIDs (just the ID part) to a JSON file

In [10]:
target_list = list(clinical_oa_df['Accession ID'])
target_list = [i[3:] for i  in target_list ]
print(target_list[-10:])
print(len(target_list))

json = json.dumps(target_list)
f = open("target_PMCs.json","w")
f.write(json)
f.close()

['5830351', '5830989', '5831655', '5831657', '5831658', '5831663', '5831898', '5832059', '5832504', '5833984']
291249


In [11]:
# sanity check for target_list
print(target_list[3])

16145


Get the list of PMCIDs of files that are currently in the directory containing the target articles for analysis

In [12]:
targetarticles_files_list = glob.glob('/home/dave/datapubmed/targetarticles/PMC*.nxml')
targetarticles_files_list = [i[40:] for i  in targetarticles_files_list  ]
targetarticles_files_list = [i[:-5] for i  in targetarticles_files_list  ]

print(targetarticles_files_list[:10])
print(len(targetarticles_files_list))

['3778263', '4473156', '5771285', '4264693', '4210730', '4578256', '5424460', '5541470', '4634690', '4548150']
291151


In [13]:
#clinical_oa_df.head(20)

#clinical_oa_df.shape

In [14]:
#df_table1 = pd.read_csv('clinical_oa_df_table1.csv')

#df_table1.shape

A bunch of lxml based functions from the PubMed Parser repo of Titipata: load the XML, parse the tables, get the article metadata

- Could go in separate .py file eventually but easier here, for now, to modify if needed

In [15]:
def load_xml(database, articleid, sleep=None):
    """
    Load XML file from given pmid from eutils site
    return a dictionary for given pmid and xml string from the site
    sleep: how much time we want to wait until requesting new xml
    """
    link = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=" + database + "&retmode=xml&id=%s" % str(articleid)
    page = requests.get(link)
    tree = html.fromstring(page.content)
    if sleep is not None:
        time.sleep(sleep)
    return tree

def parse_pubmed_web_tree(tree):
    """
    Giving tree, return simple parsed information from the tree
    """

    if tree.xpath('//articletitle') is not None:
        title = ' '.join([title.text for title in tree.xpath('//articletitle')])
    else:
        title = ''

    abstract_tree = tree.xpath('//abstract/abstracttext')
    abstract = ' '.join([stringify_children(a).strip() for a in abstract_tree])

    if tree.xpath('//article//title') is not None:
        journal = ';'.join([t.text.strip() for t in tree.xpath('//article//title')])
    else:
        journal = ''

    pubdate = tree.xpath('//pubmeddata//history//pubmedpubdate[@pubstatus="medline"]')
    if len(pubdate) >= 1 and pubdate[0].find('year') is not None:
        year = pubdate[0].find('year').text
    else:
        year = ''

    affiliations = list()
    if tree.xpath('//affiliationinfo/affiliation') is not None:
        for affil in tree.xpath('//affiliationinfo/affiliation'):
            affiliations.append(affil.text)
    affiliations_text = '; '.join(affiliations)

    authors_tree = tree.xpath('//authorlist/author')
    authors = list()
    if authors_tree is not None:
        for a in authors_tree:
            firstname = a.find('forename').text if a.find('forename') is not None else ''
            lastname = a.find('lastname').text if a.find('forename') is not None else ''
            fullname = (firstname + ' ' + lastname).strip()
            if fullname == '':
                fullname = a.find('collectivename').text if a.find('collectivename') is not None else ''
            authors.append(fullname)
        authors_text = '; '.join(authors)
    else:
        authors_text = ''

    dict_out = {'title': title,
                'abstract': abstract,
                'journal': journal,
                'affiliation': affiliations_text,
                'authors': authors_text,
                'year': year}
    return dict_out

def parse_xml_web(pmid, sleep=None, save_xml=False):
    """
    Give pmid, load and parse xml from Pubmed eutils
    if save_xml is True, save xml output in dictionary
    """
    tree = load_xml(pmid, sleep=sleep)
    dict_out = parse_pubmed_web_tree(tree)
    dict_out['pmid'] = str(pmid)
    if save_xml:
        dict_out['xml'] = etree.tostring(tree)
    return dict_out

def read_xml(path):
    """
    Parse tree from given XML path
    """
    try:
        tree = etree.parse(path)
    except:
        try:
            tree = etree.fromstring(path)
        except Exception as e:
            print("Error: it was not able to read a path, a file-like object, or a string as an XML")
            raise
    if '.nxml' in path:
        remove_namespace(tree) # strip namespace for
    return tree

def remove_namespace(tree):
    """
    Strip namespace from parsed XML
    """
    for node in tree.iter():
        try:
            has_namespace = node.tag.startswith('{')
        except AttributeError:
            continue  # node.tag is not a string (node is a comment or similar)
        if has_namespace:
            node.tag = node.tag.split('}', 1)[1]
            
def stringify_children(node):
    """
    Filters and removes possible Nones in texts and tails
    ref: http://stackoverflow.com/questions/4624062/get-all-text-inside-a-tag-in-lxml
    """
    parts = ([node.text] +
             list(chain(*([c.text, c.tail] for c in node.getchildren()))) +
             [node.tail])
    return ''.join(filter(None, parts))


def parse_article_meta(tree):
    """
    Parse PMID, PMC and DOI from given article tree
    """
    article_meta = tree.find('.//article-meta')
    pmid_node = article_meta.find('article-id[@pub-id-type="pmid"]')
    pmc_node = article_meta.find('article-id[@pub-id-type="pmc"]')
    pub_id_node = article_meta.find('article-id[@pub-id-type="publisher-id"]')
    doi_node = article_meta.find('article-id[@pub-id-type="doi"]')

    pmid = pmid_node.text if pmid_node is not None else ''
    pmc = pmc_node.text if pmc_node is not None else ''
    pub_id = pub_id_node.text if pub_id_node is not None else ''
    doi = doi_node.text if doi_node is not None else ''

    dict_article_meta = {'pmid': pmid,
                         'pmc': pmc,
                         'doi': doi,
                         'publisher_id': pub_id}

    return dict_article_meta

def table_to_df(table_text):
    """
    Function to transform plain xml text to list of row values and
    columns
    """
    table_tree = etree.fromstring(table_text)
    columns = []
    for tr in table_tree.xpath('thead/tr'):
        for c in tr.getchildren():
            columns.append(unidecode(stringify_children(c)))

    row_values = []
    len_rows = []
    for tr in table_tree.findall('tbody/tr'):
        es = tr.xpath('td')
        row_value = [unidecode(stringify_children(e)) for e in es]
        len_rows.append(len(es))
        row_values.append(row_value)
    if len(len_rows) >= 1:
        len_row = max(set(len_rows), key=len_rows.count)
        row_values = [r for r in row_values if len(r) == len_row] # remove row with different length
        return columns, row_values
    else:
        return None, None


def parse_pubmed_table(path, return_xml=True):
    """
    Parse table from given Pubmed Open-Access XML file
    """
    tree = read_xml(path)
    dict_article_meta = parse_article_meta(tree)
    pmid = dict_article_meta['pmid']
    pmc = dict_article_meta['pmc']

    # parse table
    tables = tree.xpath('//body//sec//table-wrap')
    table_dicts = list()
    for table in tables:
        if table.find('label') is not None:
            label = unidecode(table.find('label').text or '')
        else:
            label = ''

        # table caption
        if table.find('caption/p') is not None:
            caption_node = table.find('caption/p')
        elif table.find('caption/title') is not None:
            caption_node = table.find('caption/title')
        else:
            caption_node = None
        if caption_node is not None:
            caption = unidecode(stringify_children(caption_node).strip())
        else:
            caption = ''

        # table content
        if table.find('table') is not None:
            table_tree = table.find('table')
        elif table.find('alternatives/table') is not None:
            table_tree = table.find('alternatives/table')
        else:
            table_tree = None

        if table_tree is not None:
            table_xml = etree.tostring(table_tree)
            columns, row_values = table_to_df(table_xml)
            if row_values is not None:
                table_dict = {'pmid': pmid,
                              'pmc': pmc,
                              'label': label,
                              'caption': caption,
                              'table_columns': columns,
                              'table_values': row_values}
                if return_xml:
                    table_dict['table_xml'] = table_xml
                table_dicts.append(table_dict)
    if len(table_dicts) >= 1:
        return table_dicts
    else:
        return None
    
def parse_html_table_to_df(table):
    n_columns = 0
    n_rows=0
    column_names = []

    # we find the column titles if we can
    for row in table.thead.find_all('tr'):
        # Handle column names if we find them
        td_tags = row.find_all('td') 
        if len(td_tags) > 0 and len(column_names) == 0:
            for td in td_tags:
                column_names.append(td.get_text())
        
    # Find number of rows
    for row in table.tbody.find_all('tr'):
    
        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)
            
    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                      index= range(0,n_rows))
    row_marker = 0
    for row in table.tbody.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1

    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

    return df

def parse_html_table_to_json(table):
    
    rows = table.find_all("tr")

    headers = {}
    
    thead = table.find("thead")
    if thead is not None:
        ths = thead.find_all("th")

        for i in range(len(ths)):
            headers[i] = ths[i].text.strip().lower()

    data = []

    for row in rows:
        if row is not None:
            cells = row.find_all("td")

            item = {}
            if len(cells) == len(headers):

                for index in headers:
                    item[headers[index]] = unidecode(cells[index].text)

                data.append(item)

    #return json.dumps(data, indent=4)
    return data

Check the contents of any XML file in the target articles directory

In [17]:
fh = open('/home/dave/datapubmed/targetarticles/PMC4634690.nxml','r')
soup = bs.BeautifulSoup(fh,'lxml')
print(soup.prettify())
table = soup.find('table')
parse_html_table(table)

<!DOCTYPE article PUBLIC "-//NLM//DTD JATS (Z39.96) Journal Archiving and Interchange DTD v1.0 20120330//EN" "JATS-archivearticle1.dtd">
<html>
 <body>
  <article article-type="research-article" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink">
   <?properties open_access?>
   <front>
    <journal-meta>
     <journal-id journal-id-type="nlm-ta">
      EBioMedicine
     </journal-id>
     <journal-id journal-id-type="iso-abbrev">
      EBioMedicine
     </journal-id>
     <journal-title-group>
      <journal-title>
       EBioMedicine
      </journal-title>
     </journal-title-group>
     <issn pub-type="epub">
      2352-3964
     </issn>
     <publisher>
      <publisher-name>
       Elsevier
      </publisher-name>
     </publisher>
    </journal-meta>
    <article-meta>
     <article-id pub-id-type="pmid">
      26629543
     </article-id>
     <article-id pub-id-type="pmc">
      4634690
     </article-id>
     <article-id pub-id-type="publ

NameError: name 'parse_html_table' is not defined

In [18]:
def parse_html_table(table):
    n_columns = 0
    n_rows=0
    column_names = []

    # we find the column titles if we can
    for row in table.thead.find_all('tr'):
        # Handle column names if we find them
        td_tags = row.find_all('td') 
        if len(td_tags) > 0 and len(column_names) == 0:
            for td in td_tags:
                column_names.append(td.get_text())
        
    # Find number of rows
    for row in table.tbody.find_all('tr'):
    
        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)
            
    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                      index= range(0,n_rows))
    row_marker = 0
    for row in table.tbody.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1

    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

    return df

In [19]:
idlist = ['1297588']
for idee in idlist: 
    fname = glob.glob('/home/dave/datapubmed/targetarticles/PMC%s.nxml' % idee)[0]
    print(fname)
    fh = open(fname,'r')
    soup = bs.BeautifulSoup(fh,'lxml')
    table = soup.find('table')
    print(pd.DataFrame(parse_html_table(table)).iloc[:6])
    

/home/dave/datapubmed/targetarticles/PMC1297588.nxml
                               Placebo group (n = 14)  \
0                          Age            57.5 (11.0)   
1                    Sex (F:M)                    9:5   
2                  Height (cm)           168.4 (10.3)   
3       Methotrexate (yes/all)                   7/14   
4  Methotrexate dose (mg/week)             7.5 (10.1)   
5         Prednisone (yes/all)                   9/14   

  Vitamin B6 group (n = 14)  
0               53.9 (12.6)  
1                      12:2  
2               164.7 (9.1)  
3                      8/14  
4               10.2 (11.7)  
5                     11/14  


Take the list of various labels used for Table 1 in the XML and then look at which of the Table 1 include age in the cells of the table somewhere - using a regex and 'find' of Beautiful Soup. Create a dict with the information needed to reference the articles/tables later.

In [20]:
table1_list = ['1', 'T', 'TABLE', 'TABLE 1', 'TABLE 1 ', 'TABLE1', 'TABLE 1.', 'TABLE1.', 'TABLE I', 'TABLE I.',\
               'TAB. 1', 'TAB.1', 'Table', 'Table 1', 'Table 1 ', 'Table 1 -', 'Table 1.', 'Table 1:', \
              'Tab. 1', 'Tab.1', 'Table I', 'Table I.', 'Table No. 1', 'Table. 1', 'Tabela 1', 'Tableau 1']

table1_set = set(table1_list)

#regexAge = '(?:mean|M|median|Med[.])?\\W*age[d|s]?\\W*(?:mean|M)?\\W*(?:SD)?\\W*(?:range|from)?\\W+\\d+\\W*(?:to)?\\W*\\d*\\W*(?:year|day|week)'# or '\\d+[\\s\\-\\d]*(?:year[s]?|month[s]?|week[s]?)[\\s\\-]*old?'
regexAge = '(?i)(?:mean|M?)?\W*age[d|s]?\W*?(?:mean|M)?\W*(?:year[s]|day[s]|week[s])\W*(?:mean|M)?\W*[?:-|+|±]?\W*(S[.])?(D[.])?'

table_count = 0
table1_count = 0
tablecnt = []
file_list = glob.glob('/home/dave/datapubmed/targetarticles/PMC*.nxml')
choice = np.random.randint(2000, size=100)
for f in range(len(file_list)): 
#for f in choice: 

    #print(os.path.basename(file_list[f])[:-5])
    fh = open(file_list[f],'r')
    #soup = bs.BeautifulSoup(fh,'lxml')
    #print(soup.prettify())
    tree = read_xml(file_list[f])
    if tree is not None:
        dict_article_meta = parse_article_meta(tree)
        pmid = dict_article_meta['pmid'];# print (pmid)
        pmc = dict_article_meta['pmc']; #print (pmc)
        #tablelist = soup.find('table')#.findAll(text="age")
        #print(tablelist)
        tables = tree.xpath('//body//sec//table-wrap')
        if not tables:
            table_dict = {'PMCID': pmc, 'Table': 'no_tables', 'Id': '', 'Age': '', 'Sex': '', 'Gender': ''}
        else:
            table_count += 1
            for table in tables:
                if table.find('label') is not None:
                    label = unidecode(table.find('label').text or '')
                    tableid = unidecode(table.find('id') or '')
                    if label in table1_set:
                        table1_count += 1
                        soup = bs.BeautifulSoup(etree.tostring(table), "lxml")
                        #print(soup.prettify())
                        age = soup.findAll(text=re.compile(regexAge, re.I))
                        #sex = soup.findAll(text=re.compile('^Sex', re.I))
                        #gender = soup.findAll(text=re.compile('^Gender', re.I))
                        table_dict = {'PMCID': pmc, 'Table': label, 'Id': table.attrib['id'], 'Age': age, 'Sex': '', 'Gender': ''}
                        break
                    else:
                        table_dict = {'PMCID': pmc, 'Table': label, 'Id': table.attrib['id'], 'Age': '', 'Sex': '', 'Gender': ''}
                else:
                    table_dict = {'PMCID': pmc, 'Table': 'no_label', 'Id': table.attrib['id'], 'Age': '', 'Sex': '', 'Gender': ''}
        tablecnt.append(table_dict)

Check for files with corrupted article metadata so that they can be downloaded from PubMed

In [21]:
file_list = glob.glob('/home/dave/datapubmed/targetarticles/PMC*.nxml')
for f in range(len(file_list)): 
    fh = open(file_list[f],'r')
    tree = read_xml(file_list[f])
    article_meta = tree.find('.//article-meta')
    if article_meta is None:
        print(f)

Download from PubMed and missing or corrupted files

In [ ]:
getDocString3 = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id="
#really_small_list = [small_list[i] for i in range(30)]
for doc in ['216817']:
#for doc in additional_list:    
    query = getDocString3 + str(doc)
    resp = urlopen(query).read().decode('utf-8')
    f = open("/home/dave/datapubmed/targetarticles/PMC"+ str(doc) + ".nxml", "w")
    f.write(resp)
    f.close()
#soup_doc = bs.BeautifulSoup(resp,'lxml')

Check if any file has been downloaded correctly

In [22]:
tree = read_xml(file_list[216817])
print(tree.find('.//article-meta'))
print(file_list[216817])

<Element article-meta at 0x7fc948e18f88>
/home/dave/datapubmed/targetarticles/PMC5033451.nxml


-------

Print out the number of articles with a Table 1

In [23]:
print(len(tablecnt))

291151


What does the dict for the last article in the list look like - print it out 

In [24]:
tablecnt[-1]

{'Age': '',
 'Gender': '',
 'Id': '',
 'PMCID': '5337706',
 'Sex': '',
 'Table': 'no_tables'}

Print out the total number of articles that contain at least one table in HTML, and that contain a Table 1 in HTML

In [25]:
print(table_count)  # has some table
print(table1_count)  # has a Table 1

178606
174007


The number of Table 1 that contain "age"

In [26]:
withAge = [d for d in tablecnt if (d['Age'] !=  '') and (d['Age'] !=  [])]
len(withAge)

27830

Save the list of PMCIDs of articles with "age" in the Table 1 to a JSON file 

In [28]:
withAge[:10]

[{'Age': ['Age (years), median (range)'],
  'Gender': '',
  'Id': 'pone-0114008-t001',
  'PMCID': '4264693',
  'Sex': '',
  'Table': 'Table 1'},
 {'Age': ['Age (years)'],
  'Gender': '',
  'Id': 'T1',
  'PMCID': '2211287',
  'Sex': '',
  'Table': 'Table 1'},
 {'Age': ['Mean age (years)\u2009±\u2009SD'],
  'Gender': '',
  'Id': 'T1',
  'PMCID': '5767074',
  'Sex': '',
  'Table': 'Table 1'},
 {'Age': ['Mean Age (years)'],
  'Gender': '',
  'Id': 'T1',
  'PMCID': '3987498',
  'Sex': '',
  'Table': 'Table I.'},
 {'Age': ['Age (years)'],
  'Gender': '',
  'Id': 'pone.0179290.t001',
  'PMCID': '5470687',
  'Sex': '',
  'Table': 'Table 1'},
 {'Age': ['Median age, years (range)'],
  'Gender': '',
  'Id': 'T1',
  'PMCID': '3422173',
  'Sex': '',
  'Table': 'Table 1'},
 {'Age': ['Age, years'],
  'Gender': '',
  'Id': 'Tab1',
  'PMCID': '4210532',
  'Sex': '',
  'Table': 'Table 1'},
 {'Age': ['Age years'],
  'Gender': '',
  'Id': 'TB1',
  'PMCID': '4486374',
  'Sex': '',
  'Table': 'TABLE 1'},
 {

In [34]:
import json

PMCIDwithAge = [d['PMCID'] for d in withAge]
print(PMCIDwithAge[:10])
json = json.dumps(PMCIDwithAge)
f = open("table1_with_age.json","w")
f.write(json)
f.close()

['4264693', '2211287', '5767074', '3987498', '5470687', '3422173', '4210532', '4486374', '4350626', '5613850']


In [35]:
AgewithAge = [d['Age'] for d in withAge]
AgewithAgeList = []
[AgewithAgeList.append(d[0]) for d in AgewithAge]
AgewithAgeList

['Age (years), median (range)',
 'Age (years)',
 'Mean age (years)\u2009±\u2009SD',
 'Mean Age (years)',
 'Age (years)',
 'Median age, years (range)',
 'Age, years',
 'Age years',
 'Age (years, SD)',
 'Age (years) mean (SD)',
 'Age (years)',
 'Age (years)',
 'Age (years)',
 'Age [years], mean (SD)',
 'Age (years)*',
 ' Age (years)',
 'Age: mean, Years±SD',
 'Age (years)',
 'Age, years',
 'Age (years)',
 'Age (years)',
 'Age, years',
 'Age, years (mean\xa0±\xa0SD)',
 'Age [years]',
 'Age, years',
 'Mean age (years)',
 'Mean (SD) age (years)',
 'Age (years)',
 'Age (years)',
 'Age, mean (years)',
 'Age (years)',
 'Age (years)',
 'Age (years)',
 'Age [years]',
 'Age (years)',
 'Age (years)',
 'Age, years (IQR)',
 'Age (years)',
 '\xa0Body age difference from chronological age (years), mean (SD)',
 'Age (years)',
 'Age, years',
 'Age (Years) (%)',
 'Age (years), ',
 'Age (years)',
 'Age (years)',
 'Age (years)',
 '\xa0Age (years)',
 'Mean age (years)',
 'Age (years)',
 'Age (years)',
 'Age

In [36]:
from collections import Counter

ages = Counter(AgewithAgeList)
ages.most_common(40)

[('Age (years)', 13496),
 ('Age, years', 3379),
 ('Mean age (years)', 547),
 ('Age (years), mean (SD)', 409),
 ('Age [years]', 370),
 ('\u2003Age (years)', 303),
 ('Age (Years)', 226),
 ('Age, years, mean (SD)', 217),
 ('Median age, years (range)', 207),
 ('Age(years)', 204),
 ('Mean age, years (SD)', 167),
 ('Mean (SD) age (years)', 165),
 ('\u2003Age, years', 161),
 ('Maternal age (years)', 160),
 ('Median age (years)', 133),
 ('Age (years)*', 116),
 ('Gestational age (weeks)', 114),
 ('Age (years) ', 107),
 ('Mean age, years', 105),
 ('Age (years), mean ± SD', 91),
 ('\xa0Age (years)', 88),
 ('Age (years), median (IQR)', 88),
 ('Age, years (SD)', 81),
 ('Mean (SD) age, years', 81),
 ('Age years', 76),
 ('Age (years), median (range)', 73),
 ('Age (years old)', 71),
 ('Age (years):', 70),
 ('age (years)', 59),
 ('Mean age, years (range)', 57),
 ('Age (years, mean ± SD)', 57),
 ('Age, mean years (SD)', 56),
 ('Age, years, median (range)', 56),
 ('Age, years (mean ± SD)', 55),
 ('Age, y

In [37]:
idlist = ['5249075']
for idee in idlist: 
    fname = glob.glob('/home/dave/datapubmed/targetarticles/PMC%s.nxml' % idee)[0]
    print(fname)
    fh = open(fname,'r')
    soup = bs.BeautifulSoup(fh,'lxml')
    table = soup.find('table')
    dframe = pd.DataFrame(parse_html_table(table))
dframe

/home/dave/datapubmed/targetarticles/PMC5249075.nxml


,0,1,2,3,4
0,"Age, years (mean SD)",78.1 (7.2),78.1 (7.2),77.7 (7.0),77.6 (7.0)
1,"Gender, n (%)\nMale",190 (38.5),189 (39.1),210 (40.6),207 (40.8)
2,Female,303 (61.5),295 (61.0),307 (59.4),300 (59.2)
3,Body Mass Index (mean SD),27.6 (5.3),27.6 (5.3),27.7 (5.4),27.7 (5.4)
4,"Ethnic group (White British), n (%)",492 (99.8),483 (99.8),510 (98.7),500 (98.6)
5,"Self-reported arthritis, n (%)",292 (59.2),286 (59.1),300 (58),290 (57.2)
6,"Live alone, n (%)",236 (47.9),230 (47.5),220 (42.6),214 (42.2)
7,">4 prescribed medications, n (%)",313 (63.5),305 (63.0),304 (58.8),297 (58.6)
8,"Current use of foot orthoses, n (%)",191 (38.7),189 (39.1),163 (31.5),161 (31.8)
9,"1+ falls in previous 12 months, n (%)",325 (65.9),319 (65.9),332 (64.2),323 (63.7)
